In [34]:
%load_ext autoreload
%autoreload 2

from setup_imports import *  # noqa: F401,F403
from src.phrases.phrase_model import Phrase, Translation, get_phrase, get_phrase_by_english
from src.models import BCP47Language
from src.connections.gcloud_auth import get_firestore_client
from src.audio import get_voice_model, generate_translation_audio
from src.images import generate_image
from PIL import Image

# Hello World Test - Full Workflow
# =================================

OUR_PHRASE = "Hello, world!"
phrase = Phrase.create_phrase(OUR_PHRASE, source="manual")
print(f"✓ Created phrase: {phrase.phrase_hash}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
✓ Created phrase: hello_world_315f5b


In [35]:
phrase.translations

[Translation(phrase_hash='hello_world_315f5b', language=Language.make(language='en', territory='GB'), text='Hello, world!', text_lower='hello, world!', tokens=['hello', 'world'], audio=[], image=None)]

In [36]:
phrase._has_translation(BCP47Language.get("fr-FR"))

False

In [37]:

# Add French translation
fr = phrase.translate(BCP47Language.get("fr-FR"), refine=False)
print(f"✓ French: {fr.text}")


✓ French: Bonjour le monde!


In [38]:
phrase

Phrase(phrase_hash='hello_world_315f5b', english='Hello, world!', english_lower='hello, world!', tokens=['hello', 'world'], verbs=[], vocab=['hello', 'world'], source='manual', translations=[Translation(phrase_hash='hello_world_315f5b', language=Language.make(language='en', territory='GB'), text='Hello, world!', text_lower='hello, world!', tokens=['hello', 'world'], audio=[], image=None), Translation(phrase_hash='hello_world_315f5b', language=Language.make(language='fr', territory='FR'), text='Bonjour le monde!', text_lower='bonjour le monde!', tokens=['Bonjour', 'le', 'monde', '!'], audio=[], image=None)])

In [47]:
phrase.translations[1].generate_audio(context="flashcard")

Audio already exists for fr-FR flashcard normal, skipping generation
Audio already exists for fr-FR flashcard slow, skipping generation


In [48]:
phrase.translations[1]._upload_to_gcs()

✅ Authenticated with Google Cloud project: swedish-course
✅ Google Cloud Storage client initialized


ValueError: No image attached to translation (fr-FR)

In [20]:

# Add German translation
de = phrase.translate(BCP47Language.get("de-DE"), refine=True)
print(f"✓ German: {de.text}")


✓ German: Hallo, Welt!


In [ ]:
# Generate audio for each translation using the new generate_audio() method
# For flashcard context: generates slow and normal speeds
# For story context: generates normal and fast speeds

# French audio
print("Generating French audio (flashcard)...")
fr.generate_audio(context="flashcard", bucket_name="audio-language-trainer-private-content")
print(f"✓ French audio generated: {len(fr.audio)} audio files")
for audio in fr.audio:
    print(f"  - {audio.context} / {audio.speed}: {audio.file_path}")

# German audio
print("\nGenerating German audio (flashcard)...")
de.generate_audio(context="flashcard", bucket_name="audio-language-trainer-private-content")
print(f"✓ German audio generated: {len(de.audio)} audio files")
for audio in de.audio:
    print(f"  - {audio.context} / {audio.speed}: {audio.file_path}")

In [ ]:
# Get the English translation and generate image
en_gb = phrase.translations[0]  # The en-GB translation created in create_phrase()
print(f"✓ English translation: {en_gb.text}")

# Generate image for the English translation (shared across all language variants)
image_prompt = "A cheerful greeting scene with two people waving hello, bright morning sunlight"
image = generate_image(image_prompt, style="ghibli", project_id="swedish-course")
en_gb.image = image
en_gb.upload_image("audio-language-trainer-private-content")
print(f"✓ Image generated and attached: {en_gb.image_file_path}")

In [50]:
# Test phrase generation with small vocab_dict
from src.phrases.generation import generate_phrases_from_vocab_dict

# Small test vocab_dict with a few verbs and vocabs
test_vocab_dict = {
    "verbs": ["want", "go", "see"],
    "vocab": ["apple", "table", "red", "big", "old", "door", "window", "house"]
}

print("=" * 60)
print("TESTING PHRASE GENERATION")
print("=" * 60)
print(f"\nInput vocab_dict:")
print(f"  Verbs: {test_vocab_dict['verbs']}")
print(f"  Vocab: {test_vocab_dict['vocab']}")
print(f"\nGenerating phrases...")
print("-" * 60)

try:
    phrases, tracking = generate_phrases_from_vocab_dict(
        test_vocab_dict, 
        max_iterations=1  # Just one iteration for testing
    )
    
    print(f"\n✓ Generation complete!")
    print(f"\nResults:")
    print(f"  Total phrases generated: {tracking['total_phrases']}")
    print(f"  Verb phrases: {tracking['verb_phrases']}")
    print(f"  Vocab phrases: {tracking['vocab_phrases']}")
    print(f"  Verbs processed: {tracking['verbs_processed']}")
    print(f"  Vocab processed: {tracking['vocab_processed']}")
    print(f"  Additional words found: {len(tracking['words_used'])}")
    
    if tracking['errors']:
        print(f"\n⚠ Errors encountered:")
        for error in tracking['errors']:
            print(f"  - {error}")
    
    print(f"\nGenerated Phrases (first 20):")
    for i, phrase in enumerate(phrases[:20], 1):
        print(f"  {i}. {phrase}")
    
    if len(phrases) > 20:
        print(f"  ... and {len(phrases) - 20} more phrases")
    
    print(f"\nAdditional words tracked:")
    print(f"  {tracking['words_used']}")
    
except Exception as e:
    print(f"\n✗ Error during phrase generation:")
    print(f"  {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()

TESTING PHRASE GENERATION

Input vocab_dict:
  Verbs: ['want', 'go', 'see']
  Vocab: ['apple', 'table', 'red', 'big', 'old', 'door', 'window', 'house']

Generating phrases...
------------------------------------------------------------
Starting verb phrase generation. 3 verbs to process.
  [1/3] Generating phrases for verb: 'want'
  ERROR: Error generating phrases for verb 'want': Failed to generate phrases for verb 'want': Error code: 404 - {'type': 'error', 'error': {'type': 'not_found_error', 'message': 'model: claude-sonnet-latest'}, 'request_id': 'req_011CVR6zTHysr5DCibt3LDrf'}
  [2/3] Generating phrases for verb: 'go'
  ERROR: Error generating phrases for verb 'go': Failed to generate phrases for verb 'go': Error code: 404 - {'type': 'error', 'error': {'type': 'not_found_error', 'message': 'model: claude-sonnet-latest'}, 'request_id': 'req_011CVR6zU38TNCN3w98d1XqT'}
  [3/3] Generating phrases for verb: 'see'
  ERROR: Error generating phrases for verb 'see': Failed to generate phr